### Tóm tắt

Chúng tôi đề xuất xây dựng hệ thống rút tin tức từ Internet và thống kê các từ khóa chính xuất hiện theo thời gian thực. Cơ sở dữ liệu căn bản của chúng tôi dùng là Postgresql, kết hợp với các tiện ích khác để xây dựng hệ thống lưu trữ phân tán và cơ sở dữ liệu chuỗi thời gian giúp đáp ứng được các yêu cầu cơ bản. Hệ thống được đề xuất tuy đơn giản nhưng có khả năng đáp ứng được các yêu cầu của dữ liệu lớn và dễ dàng mở rộng khi cần.

### Giới thiệu

Rút tin (Web scraping) là quá trình xử lý, rút trích thông tin tự động từ các trang trên Internet để tổng hợp và làm giàu nguồn dữ liệu cho chính ứng dụng hiện tại. Một hệ thống rút tin đơn giản bao gồm frontier chứa các url cần được xử lý. Các url sẽ được xử lý tuần tự hoặc theo một nguyên tắc nào đó và rút các nội dung từ thẻ html. Các nội dung sẽ được thêm vào một cơ sở dữ liệu lưu trữ cùng với các url mới được tiếp tục thêm vào frontier. 

CSDL quan hệ (SQL) được ra mắt từ rất sớm và đem lại nhiều lợi ích cho con người. Nhưng khi nhu cầu khai thác dữ liệu tăng cao, các CSDL phi quan hệ (NoSQL) khi đó ra đời như MapReduce (1), Google Bigtable (2), Cassandra (3), Mongodb (4),... đem lại nhiều tiện ích khiến chúng ta dần quên đi SQL trong các ứng dụng. Phải mất một thời gian, các CSDL quan hệ được mới tiếp tục được sử dụng nhiều trở lại bởi các dự án mã nguồn mở đã làm cho chúng ngày càng mềm dẻo. 


Trong nhiều trường hợp, việc sử dụng các NoSQL là không cần thiết và thậm chí còn gây lãng phí bởi các CSDL quan hệ ngày nay đã đáp ứng được các dữ liệu phi cấu trúc. Postgresql sử dụng những kiểu dữ liệu cơ bản của SQL thông thường cộng thêm một số kiểu dữ liệu phi cấu trúc làm cho nó trở nên tiện dụng. Ngoài ra, các tiện ích mở rộng cũng như bản quyền Postgres đã giúp cho nó phát triển nhanh và trở thành CSDL được yêu thích nhất năm 2019 (5). Chính vì các điểm trên, chúng tôi đề xuất sử dụng Postgresql làm cơ sở dữ liệu lưu trữ thông tin cho ứng dụng.  

Các điểm nhấn của đồ án cuối kỳ như sau: 

1. Xây dựng hệ thống rút tin tức và thống kê từ khóa đơn giản nhưng đáp ứng được luồng dữ liệu đến cực lớn và có khả năng mở rộng linh hoạt  
2. Đề xuất sử dụng cơ sở dữ liệu (CSDL) quan hệ và các tiện ích mở rộng để lưu trữ và thống kê dữ liệu thay cho các CSDL phi quan hệ (NoSQL)
3. Triển khai các dịch vụ trên docker 

### Hệ thống 

Chúng tôi đề xuất sử dụng Postgresql (Postgres), Redis, Kafka cùng một số tiện ích mở rộng của Postgres làm cơ sở xây dựng hệ thống. Hình bên dưới minh họa tóm tắt hệ thống chúng tôi xây dựng.

<img src="images/system.png">

Hệ thống trên gồm hai thành phần chính. ```www``` có nhiệm vụ thu thập các url từ những trang web cho trước. Các url được chuẩn hóa và lưu trữ dưới dạng mã hóa sha1 trong Redis một bản duy nhất (với key là mã băm của url). Với những url mới, ```www``` đưa chúng vào Kafka và chờ chương trình rút nội dung, rút thực thể xử lý. 

Vì các bài báo mới không nhiều nên chúng tôi đưa chúng trực tiếp vào Postgres. Các keywords xuất hiện nhiều nên chúng tôi đưa chúng vào Kafka và dùng tiện ích mở rộng khác của Postgres là PipelineDB + PipelineKafka để đồng bộ số dữ liệu này. Các từ khóa sẽ được thống kê thông qua khung nhìn liên tục của PipelineDB được viết giống khung nhìn của SQL cơ bản. 

Để phân tán dữ liệu, chúng tôi sử dụng Citusdata với master và các worker node. Bảng lưu trữ dữ liệu sẽ được tạo thành bảng phân tán thông qua lệnh ```create_distributed_table``` của Citus. Vì phần cứng hạn chế nên chúng tôi chỉ thực hiện khởi tạo 1 master node và 3 worker tương ứng với master và slave 1 - 3. 

Chúng tôi thực hiện cài đặt các phần mềm cơ sở dữ liệu trực tiếp trên các slave master và slave 1 - 3. Các dịch vụ, mã nguồn khác được chạy độc lập trên container ở slave 4

### Cài đặt 

#### Cài đặt docker và docker-compose trên slave 4 

Cài đặt các gói cần thiết

```sudo apt install apt-transport-https ca-certificates curl software-properties-common```

Tiếp theo, cập nhật key của docker vào repository của ubuntu

```curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -```

Thêm vào apt source list

```sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"```

Cập nhật repository 

```sudo apt-get update```

Cài đặt docker community edition

```sudo apt install -y docker-ce```

##### Cài đặt Postgresql tại master và các slave còn lại 

Thêm key postgresql 

```wget -q https://www.postgresql.org/media/keys/ACCC4CF8.asc -O- | sudo apt-key add -```

Thêm vào source list 

```echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" | sudo tee /etc/apt/sources.list.d/postgresql.list```

Cập nhật repository

```sudo apt-get update```

Cài đặt postges 11 

```sudo apt-get install -y postgresql-11```

#### Citusdata tại master và các slave còn lại 

Thêm vào repository

```curl https://install.citusdata.com/community/deb.sh | sudo bash```

Cài Citus

```sudo apt-get -y install postgresql-11-citus```

Sau khi cài đặt xong Postgres và Citus ở các máy, tiếp tục thực hiện mở địa chỉ để các node Postgres giao tiếp với nhau. Thực hiện các bước sau trên tất cả các máy này: 

1. Chỉnh listen 

```sudo vim /etc/postgresql/11/main/postgresql.conf``` 

tìm dòng ```listen_addresses='127.0.0.1``` và thay bằng ```listen_addresses='*'```

2. Tiếp theo thêm extension bằng cách thêm dòng  

```shared_preload_libraries = 'citus'```

3. Lưu và tiếp tục sửa 

```sudo vim /etc/postgresql/11/main/pg_hba.conf``` 

thay 

```host    all             all             127.0.0.1/32              md5```

thành 

```host    all             all             0.0.0.0/0              trust```


4. Lưu và chạy lệnh khởi động lại pg server 

```service postgresql restart``` 

** Kiểm tra xem pg còn hoạt động hay không 


```service postgresql status```

và cluster 

```pg_lsclusters```


#### Citusdata tại master node

Chọn csdl và tạo extension bằng lệnh 

```CREATE EXTENSION citus;```

Kết nối với các slave bằng lệnh 

```SELECT * from master_add_node(SLAVE_IP_ADDRESS_OR_NAME, BINDING_PORT);```

ví dụ: 

```SELECT * from master_add_node('slave-1', 5432);```

#### Cài PipelineDB tại master

Thêm apt repository 

```curl -s http://download.pipelinedb.com/apt.sh | sudo bash```

Cài đặt 

```sudo apt-get install pipelinedb-postgresql-11```

Chỉnh preload của pg trong tập tin ```sudo vim /etc/postgresql/11/main/postgresql.conf```

```shared_preload_libraries = 'citus,pipelinedb'```

và 

```max_worker_processes = 128```

khởi động lại Postgres server bằng lệnh ```service postgresql restart``` 

#### Cài kafka-pipeline 

Thực hiện lần lượt các lệnh sau:

```sudo apt-get install git gcc g++ zlib1g-dev```

```git clone https://github.com/edenhill/librdkafka.git```

```git clone https://github.com/pipelinedb/pipeline_kafka.git```

```cd librdkafka```

```./configure --prefix=/usr ```

```make && sudo make install```


```cd pipeline_kafka```

```./configure ```

```make && sudo make install```

Cuối cùng, chỉnh preload trong postgresql.conf và khởi động lại server 

```shared_preload_libraries = 'citus,pipelinedb,kafka-pipeline'```

#### Khởi động các container app tại slave 4 

Di chuyển đến thư mục project và gõ lệnh:

```sudo docker-compose up -d```

### Chi tiết hiện thực

Chúng tôi hiện thực chương trình bằng python3.6 được chạy trên ubuntu 18.04. Chúng tôi thực hiện kết nối với Postgresql bằng ```sqlalchemy```, thực hiện các thao tác với Kafka bằng ```kafka-python```. 

Các bài báo được rút từ 150 nguồn tiếng Anh (xem ```pages.txt```). Chúng tôi thực hiện rút tiêu đề, tóm tắt, nội dung, ngày đăng, tác giả từ bài báo thông qua thư viện ```newspaper3k```. Từ tiêu đề và nội dung, chúng tôi tiếp tục thực hiện rút các thực thể, hay còn gọi là từ khóa bằng thư viện ```nltk```. Các từ khóa sẽ được thống kê số lượng xuất hiện mỗi ngày bằng khung nhìn liên tục của PipelineDB từ bảng stream. Khung nhìn liên tục được viết bằng lệnh sql: 



```CREATE VIEW keyword_stats WITH (action=materialize) AS
SELECT data.keyword, DATE(data.created_date) AS date, COUNT(DATE(data.created_date)) AS total
FROM public.data
GROUP BY data.keyword, DATE(data.created_date)
```

Mã nguồn đồ án nằm trong thư mục ```src``` gồm các tập tin:

1. ```arguments.py``` chứa các tham số khi chạy chương trình như sau: 

```--kafka_host``` địa chỉ host và port để kết nối đến kafka server. Ví dụ: 127.0.0.1:9092 

```--kafka_link_topic``` topic chứa các liên kết, dùng như hàng đợi chuẩn bị cho rút nội dung 

```--kafka_keyword_topic``` topic chứa các keyword sau khi đã được rút nội dung. PipelineDB sẽ liên kết đến topic này và tự động stream dữ liệu và bảng chuẩn bị cho thống kê 

```--kafka_default_group``` nhóm làm việc của kafka consumer, dùng để theo dõi những offset đã duyệt qua để tiếp tục chạy phòng trường hợp consumer bị ngắt 

```--redis_host``` địa chỉ host của redis database 

```--redis_port``` port redis 

```--redis_db``` số thứ tự của redis database. Mặc định ở đây là $1$ 

```--redis_password``` mật khẩu của redis server 

```--pg_host``` địa chỉ host của postgresql

```--pg_port``` port postgres

```--pg_user``` user được quyền truy cập vào database 

```--pg_password``` password của user 

```--pg_db``` database cần được truy cập 

```--pg_relation``` tên bảng chứa dữ liệu tin được rút về 




2. ```helper.py``` hiện thực các phương thức như kết nối database, database model, rút thực thể (keyword) từ text, mã hóa url, chuẩn hóa url 


3. ```logger.py``` ghi log của chương trình 

4. ```visitor.py``` lấy các link nội dung từ url cho trước và gởi vào Kafka 

5. ```scraper.py``` rút nội dung từ các link trong Kafka topic. Sau khi thực hiện xong, nội dung bài báo sẽ được gởi thẳng vào Postgresql. Các keywords được gởi vào PipelineDB thông qua Kafka

*** Ví dụ chạy xem trong tập tin ```runvisitor.sh``` và ```runscraper.sh```

### Kết quả

### Tham khảo

Cài đặt docker: https://www.digitalocean.com/community/tutorials/how-to-install-and-use-docker-on-ubuntu-18-04

Docker Kafka và Zookeeper: https://github.com/wurstmeister/kafka-docker

Docker Redis: https://hub.docker.com/_/redis

Cài đặt Postgres 11: https://www.itzgeek.com/how-tos/linux/ubuntu-how-tos/how-to-install-postgresql-10-on-ubuntu-18-04-lts.html

Citus data: http://docs.citusdata.com/

PipelineDB: http://docs.pipelinedb.com/

